In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Home 4: Build a seq2seq model for machine translation.

### Name: [Jhao-Han Chen]

### Task: Translate English to [Spanish]

## 0. You will do the following:

1. Read and run my code.
2. Complete the code in Section 1.1 and Section 4.2.

    * Translation English to **German** is not acceptable!!! Try another language.
    
3. **Make improvements.** Directly modify the code in Section 3. Do at least one of the followings. By doing more, you will get up to 2 bonus scores to the total.

    * Bi-LSTM instead of LSTM
    
    * Multi-task learning (e.g., both English to French and English to Spanish)
    
    * Attention
    
4. Evaluate the translation using the BLEU score. 

    * Optional. Up to 1 bonus scores to the total.
    
5. Convert the notebook to .HTML file. 

    * The HTML file must contain the code and the output after execution.

6. Put the .HTML file in your own Github repo. 

7. Submit the link to the HTML file to Canvas.    


### Hint: 

To implement ```Bi-LSTM```, you will need the following code to build the encoder; the decoder won't be much different.

In [0]:
from keras.layers import Bidirectional, Concatenate

encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

### Hint: 

To implement multi-task training, you can refer to ```Section 7.1.3 Multi-output models``` of the textbook, ```Deep Learning with Python```.

## 1. Data preparation

1. Download data (e.g., "deu-eng.zip") from http://www.manythings.org/anki/
2. Unzip the .ZIP file.
3. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".

### 1.1. Load and clean text


In [0]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [0]:
# e.g., filename = 'Data/deu.txt'
filename = '/content/drive/My Drive/spa.txt'

# e.g., n_train = 20000
n_train = 100000

In [0]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [5]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[i loved that] => [eso me gusto]
[i loved that] => [eso me gustaba]
[i loved that] => [a mi me gustaba eso]
[i made a bet] => [hice una apuesta]
[i made these] => [yo hice estos]
[i made these] => [estos los hice yo]
[i misled tom] => [yo engane a tom]
[i missed you] => [te eche de menos]
[i missed you] => [te extrane]
[i need a cab] => [necesito un taxi]


In [6]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (100000,)
Length of target_texts: (100000,)


In [7]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 41
max length of target sentences: 75


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

Using TensorFlow backend.


shape of encoder_input_seq: (100000, 41)
shape of input_token_index: 27
shape of decoder_input_seq: (100000, 75)
shape of target_token_index: 29


In [9]:

num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [10]:
target_texts[100]

'\tsed buenas\n'

In [11]:
decoder_input_seq[100, :]

array([12,  5,  2, 14,  1, 18, 11,  2,  6,  3,  5, 13,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0], dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [12]:
from keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(100000, 41, 28)
(100000, 75, 30)


## 3. Build the networks (for training)

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.

### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [13]:
from keras.layers import Bidirectional, Concatenate
from keras.layers import Input, LSTM
from keras.models import Model

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                    dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Print a summary and save the encoder network structure to "./encoder.pdf"

In [14]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.png'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 28)     0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 583680      encoder_inputs[0][0]             
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 512)          0           bidirectional_1[0][2]      

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [0]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(2*latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(2*latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h_, state_c_ = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])
# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h_, state_c_],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [16]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.png'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input_x (InputLayer)    (None, None, 30)     0                                            
__________________________________________________________________________________________________
decoder_input_h (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_input_c (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1112064     decoder_input_x[0][0]            
                                                                 decoder_input_h[0][0]      

### 3.3. Connect the encoder and decoder

In [0]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [16]:
print(state_h)
print(decoder_input_h)

Tensor("concatenate_1/concat:0", shape=(?, 512), dtype=float32)
Tensor("decoder_input_h:0", shape=(?, 512), dtype=float32)


In [19]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.png'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_x (InputLayer)    (None, None, 28)     0                                            
__________________________________________________________________________________________________
decoder_input_x (InputLayer)    (None, None, 30)     0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 512), (None, 583680      encoder_input_x[0][0]            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1112064     decoder_input_x[0][0]            
                                                                 encoder[1][0]       

### 3.5. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [0]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(100000, 41, 28)
shape of decoder_input_data(100000, 75, 30)
shape of decoder_target_data(100000, 75, 30)


In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

#model.save('/content/drive/My Drive/seq2seq1.h5')

Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 333s 4ms/step - loss: 0.7914 - val_loss: 0.8922
Epoch 2/50
80000/80000 [==============================] - 325s 4ms/step - loss: 0.5733 - val_loss: 0.7586
Epoch 3/50
80000/80000 [==============================] - 330s 4ms/step - loss: 0.4917 - val_loss: 0.6644
Epoch 4/50
80000/80000 [==============================] - 332s 4ms/step - loss: 0.4410 - val_loss: 0.6152
Epoch 5/50
80000/80000 [==============================] - 340s 4ms/step - loss: 0.4057 - val_loss: 0.5762
Epoch 6/50
80000/80000 [==============================] - 339s 4ms/step - loss: 0.3804 - val_loss: 0.5518
Epoch 7/50
80000/80000 [==============================] - 337s 4ms/step - loss: 0.3619 - val_loss: 0.5316
Epoch 8/50
80000/80000 [==============================] - 335s 4ms/step - loss: 0.3466 - val_loss: 0.5179
Epoch 9/50
80000/80000 [==============================] - 335s 4ms/step - loss: 0.3339 - val_loss: 0.5

## 4. Make predictions


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [0]:
model.load_weights('/content/drive/My Drive/seq2seq1.h5')

In [0]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [22]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


-
English:        is he right
Spanish (true):  esta el bien
Spanish (pred):  esta rico
-
English:        is he right
Spanish (true):  se encuentra bien
Spanish (pred):  esta rico
-
English:        is he right
Spanish (true):  ha acertado
Spanish (pred):  esta rico
-
English:        is he right
Spanish (true):  esta en lo cierto
Spanish (pred):  esta rico
-
English:        is he right
Spanish (true):  es lo que el dice
Spanish (pred):  esta rico
-
English:        is it clean
Spanish (true):  esta limpio
Spanish (pred):  esta limpio
-
English:        is it clean
Spanish (true):  esta limpia
Spanish (pred):  esta limpio
-
English:        is it there
Spanish (true):  esta ahi
Spanish (pred):  esta ahi
-
English:        is it to go
Spanish (true):  para llevar
Spanish (pred):  es importante
-
English:        is it windy
Spanish (true):  esta ventoso
Spanish (pred):  esta bien
-
English:        is it yours
Spanish (true):  es suyo
Spanish (pred):  es suyo
-
English:        is it yours
Spanis

### 4.2. Translate an English sentence to the target language

1. Tokenization
2. One-hot encode
3. Translate

In [23]:
input_sentence = 'why is that'

input_sequence1=[]
tokenizer = Tokenizer(char_level=True, filters='')
tokenizer.fit_on_texts(input_texts)
seqs1 = tokenizer.texts_to_sequences(input_sentence)
for sublist in seqs1:
    for item in sublist:
        input_sequence1.append(item)
input_sequence = pad_sequences([input_sequence1], maxlen=max_encoder_seq_length, padding='post')

input_x = onehot_encode(input_sequence, max_encoder_seq_length, num_encoder_tokens)
#<do one-hot encode...>

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

source sentence is: why is that
translated sentence is: por que es eso



## 5. Evaluate the translation using BLEU score

Reference: 
- https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
- https://en.wikipedia.org/wiki/BLEU


**Hint:** 

- Randomly partition the dataset to training, validation, and test. 

- Evaluate the BLEU score using the test set. Report the average.

- A reasonable BLEU score should be 0.1 ~ 0.3.

In [0]:
import pandas as pd
df=pd.DataFrame(clean_pairs, columns=['English','Spanish','0'])
del df['0']

In [18]:
df.head()

,English,Spanish
0,go,ve
1,go,vete
2,go,vaya
3,go,vayase
4,hi,hola


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.English, df.Spanish, test_size=0.2, random_state=1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1)

In [20]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(72000,)
(8000,)
(20000,)
(72000,)
(8000,)
(20000,)


In [0]:
y_train = ['\t' + text + '\n' for text in y_train]
y_val = ['\t' + text + '\n' for text in y_val]
y_test = ['\t' + text + '\n' for text in y_test]

In [0]:
def seq_onehoc(data,input,maxlen,num):
  tokenizer = Tokenizer(char_level=True, filters='')
  tokenizer.fit_on_texts(input)
  seqs=tokenizer.texts_to_sequences(data)
  input_sequence=pad_sequences(seqs, maxlen=maxlen, padding='post')
  input_x = onehot_encode(sequences=input_sequence, max_len=maxlen, vocab_size=num)
  return input_x, input_sequence

In [0]:
input_x_train=seq_onehoc(data=x_train,input=input_texts,maxlen=max_encoder_seq_length,num=num_encoder_tokens)[0]
input_x_val=seq_onehoc(data=x_val,input=input_texts,maxlen=max_encoder_seq_length,num=num_encoder_tokens)[0]
input_x_test=seq_onehoc(data=x_test,input=input_texts,maxlen=max_encoder_seq_length,num=num_encoder_tokens)[0]
input_y_train=seq_onehoc(data=y_train,input=target_texts, maxlen=max_decoder_seq_length,num=num_decoder_tokens)[0]
input_y_val=seq_onehoc(data=y_val,input=target_texts,maxlen=max_decoder_seq_length,num=num_decoder_tokens)[0]
input_y_test=seq_onehoc(data=y_test,input=target_texts,maxlen=max_decoder_seq_length,num=num_decoder_tokens)[0]

In [0]:
a=seq_onehoc(data=y_train,input=target_texts, maxlen=max_decoder_seq_length,num=num_decoder_tokens)[1]
decoder_target_seq = numpy.zeros(a.shape)
decoder_target_seq[:, 0:-1] = a[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

In [0]:
b=seq_onehoc(data=y_val,input=target_texts,maxlen=max_decoder_seq_length,num=num_decoder_tokens)[1]
decoder_target_seq1 = numpy.zeros(b.shape)
decoder_target_seq1[:, 0:-1] = b[:, 1:]
decoder_target_data1 = onehot_encode(decoder_target_seq1, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

In [0]:
latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                    dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')

In [0]:
# inputs of the decoder network
decoder_input_h = Input(shape=(2*latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(2*latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h_, state_c_ = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])
# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h_, state_c_],
                      name='decoder')

In [0]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [32]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([input_x_train, input_y_train], decoder_target_data, epochs=50, validation_data=([input_x_val, input_y_val], decoder_target_data1))
#model.save('/content/drive/My Drive/seq2seq2.h5')

Train on 72000 samples, validate on 8000 samples
Epoch 1/50
72000/72000 [==============================] - 501s 7ms/step - loss: 0.6405 - val_loss: 0.5183
Epoch 2/50
72000/72000 [==============================] - 496s 7ms/step - loss: 0.5296 - val_loss: 0.4360
Epoch 3/50
72000/72000 [==============================] - 496s 7ms/step - loss: 0.4714 - val_loss: 0.3962
Epoch 4/50
72000/72000 [==============================] - 494s 7ms/step - loss: 0.4357 - val_loss: 0.3706
Epoch 5/50
72000/72000 [==============================] - 495s 7ms/step - loss: 0.4106 - val_loss: 0.3494
Epoch 6/50
72000/72000 [==============================] - 495s 7ms/step - loss: 0.3922 - val_loss: 0.3365
Epoch 7/50
72000/72000 [==============================] - 493s 7ms/step - loss: 0.3773 - val_loss: 0.3241
Epoch 8/50
72000/72000 [==============================] - 493s 7ms/step - loss: 0.3654 - val_loss: 0.3157
Epoch 9/50
72000/72000 [==============================] - 493s 7ms/step - loss: 0.3552 - val_loss: 0.30

In [0]:
x_test_predict=[]
for i in range(20000):
  x_test_predict.append(decode_sequence(input_x_test[i:i+1])[0:-1])

In [37]:
x_test_predict[:5]

['tom no trabaja trabajo',
 'necesitamos un diccionario es un extrano',
 'no he hacho nada',
 'su hermano mayor le perdio la mesa',
 'quien sabe nada de un problema sentido']

In [0]:
df_test=pd.DataFrame(x_test)

In [0]:
df_test['predict_Spanish']=x_test_predict
df_test['Spanish']=y_test
df_test['Spanish']=df_test['Spanish'].apply(lambda x: x[1:-1])


In [40]:
from nltk.translate.bleu_score import sentence_bleu
bleu_score=[]
for i in range(20000):
  bleu_score.append(sentence_bleu([df_test.iloc[i,2]],df_test.iloc[i,1]))
df_test['Bleu score']=bleu_score


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [41]:
df_test.head()

,English,predict_Spanish,Spanish,Bleu score
43660,tom doesnt work anymore,tom no trabaja trabajo,tom ya no trabaja,0.596162
87278,we need a doctor its an emergency,necesitamos un diccionario es un extrano,necesitamos un medico es una emergencia,0.581506
14317,ive done nothing,no he hacho nada,no hice nada,0.416939
81932,his brother passed away last month,su hermano mayor le perdio la mesa,su hermano ha muerto el mes pasado,0.484952
95321,who doesnt know such a simple proverb,quien sabe nada de un problema sentido,quien no conoce un proverbio tan simple,0.373444


In [42]:
BLEU_mean=df_test['Bleu score'].mean()
print(f'BLEU Score: {BLEU_mean} ')

BLEU Score: 0.46776780196431644 
